In [1]:
import os

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.part_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：10


['CCR-3',
 'CCR-6',
 'CCR-7',
 'DS-2282',
 'LS-312',
 'NS-35',
 'NS-56',
 'NS-63',
 'NS-68',
 'NS-69']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part={input_.loc[a, 'Part Number']}&func=PART&vid='''
        check_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            dict_dict = resp.json()
            if dict_dict:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
dict_attr, dict_ = {}, dict_dict['pd']
for key, value in dict_.items():
    if key not in ['rowId', 'basePart', 'maxRows'] and value != None:
        if type(value) in [int, float]:
            dict_attr[key.strip().title()] = value
        elif type(value) == str:
            dict_attr[key.strip().title()] = value.strip()
        else:
            print('fuck')

dict_attr

{'Site': 'STD',
 'Webbase': 'CCR3',
 'Brand': 'STI',
 'Partdesc_En': 'Clutch Starter Safety / Cruise Release Switch',
 'Partlongdesc_En': 'Clutch Starter Safety / Cruise Release Switch',
 'Pop': 'F',
 'Qtyeach': 1}

In [9]:
vehicle = '\n'.join([dict_vehicle['buyersGuideDesc'].strip() for dict_vehicle in dict_dict['pp']['buyersGuides']])

print(vehicle)

Kia Sephia (01-94)
Kia Spectra (04-00)
Kia Sportage (00-95)


In [10]:
list_spec, dict_count = dict_dict['pp']['partSpecs'], {}
for i, spec in enumerate(list_spec):
    key = spec['attributeName_en'].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_spec, dict_no = {}, {}
for i, spec in enumerate(list_spec):
    key = spec['attributeName_en'].strip()
    if dict_count[key] == 1:
        dict_spec[str(i)] = {key: spec['siteValue_en'].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_spec[str(i)] = {f'''{key}-{dict_no[key]}''': spec['siteValue_en'].strip()}

dict_spec

{'0': {'Warranty': '3 years/36,000 mile'},
 '1': {'Connector Gender': 'Female'},
 '2': {'Terminal Quantity': '2'},
 '3': {'Terminal Gender': 'Male'},
 '4': {'Terminal Type': 'Blade'}}

In [11]:
service = '\n'.join([f'''{i+1}. {service['serviceName'].strip()}''' for i, service in enumerate(dict_dict['pp']['completeRepair']['serviceSpec'])])

print(service)

1. Fuel Filter Flush (Required)
2. Oil Change (Required)


In [12]:
import json

df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
df_temp.loc[0, 'Vehicle'] = vehicle
df_temp.loc[0, 'Service'] = service
df_temp = pd.concat([df_temp, pd.DataFrame([dict_attr])], axis=1).fillna('')
df_temp.loc[0, 'Json_Spec'] = json.dumps(dict_spec)

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...,Kia Sephia (01-94)\nKia Spectra (04-00)\nKia S...,1. Fuel Filter Flush (Required)\n2. Oil Change...,STD,CCR3,STI,Clutch Starter Safety / Cruise Release Switch,Clutch Starter Safety / Cruise Release Switch,F,1,"{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...,Kia Sephia (01-94)\nKia Spectra (04-00)\nKia S...,1. Fuel Filter Flush (Required)\n2. Oil Change...,STD,CCR3,STI,Clutch Starter Safety / Cruise Release Switch,Clutch Starter Safety / Cruise Release Switch,F,1,"{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
